In [1]:
import pandas as pd
import duckdb
import pymysql
from sqlalchemy import create_engine

In [2]:
user = 'root'
password = 'Anusha21'
host = 'localhost'
port = 3306
database = 'sales'

In [3]:
def get_connection():
    return create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )

In [4]:

engine = get_connection()
conn_alchemy=engine.connect()
con=duckdb.connect()

In [5]:

df01 =pd.read_sql("SELECT * FROM sales.address",conn_alchemy)
df01

,CustomerID,FirstName,LastName,Address,City
0,R2001,Sudhanshu,Sharma,"12, MG Road, Sector 49",Noida
1,R2002,Kapil,Khanna,"12/5, RR Marg, Bandra",Mumbai
2,R2003,Megha,Raj,"12, T Nagar",Chennai
3,R2004,David,John,"24, George Nagar",Hyderabad
4,R2005,William,James,"6, Rao Street",Kolkotta
...,...,...,...,...,...
69,R2070,Raj,Sharma,"7, EM Road",Mysore
70,R2071,Daniel,Jacob,"3 c, Ram Marg",Gandhinagar
71,R2072,Reema,Gupta,"1/14, BB Complex",Patna
72,R2073,Roshan,Rai,"12, Pacific Avenue",Bangalore


In [6]:
df02 = pd.read_sql("SELECT * FROM sales.orders",conn_alchemy)

In [7]:
df02.head()

,CustomerID,TypeOfOrder,OrderAmount,OrderDate,DeliveryDate
0,R2001,Retail,3000000,2022-04-03,2022-04-03
1,R2002,Wholesale,30000000,2022-02-01,2022-02-01
2,R2003,Overseas,200000,2022-04-03,2022-04-05
3,R2004,Wholesale,50000000,2022-04-02,2022-04-06
4,R2005,Retail,4000000,2022-02-02,2022-02-02


In [8]:
con.register('address',df01)
con.register('orders',df02)

QUERY-1

In [9]:
df1=con.sql("""SELECT week(OrderDate) AS WeekNumber,TypeofOrder,
Sum(OrderAmount) as SalesRevenue FROM orders
GROUP BY week(OrderDate) ,TypeOfOrder;""")
df1

┌────────────┬─────────────┬──────────────┐
│ WeekNumber │ TypeOfOrder │ SalesRevenue │
│   int64    │   varchar   │    int128    │
├────────────┼─────────────┼──────────────┤
│         17 │ Wholesale   │     90000000 │
│          7 │ Retail      │       200000 │
│         11 │ Wholesale   │       200000 │
│         16 │ Overseas    │      4000000 │
│          8 │ Retail      │      4000000 │
│         23 │ Overseas    │    160000000 │
│         21 │ Overseas    │      6000000 │
│          6 │ Overseas    │     80000000 │
│         28 │ Overseas    │     60000000 │
│         13 │ Wholesale   │    150000000 │
│          · │     ·       │          ·   │
│          · │     ·       │          ·   │
│          · │     ·       │          ·   │
│         32 │ Wholesale   │       400000 │
│         38 │ Wholesale   │      3000000 │
│         13 │ Overseas    │       200000 │
│         15 │ Overseas    │     80000000 │
│         18 │ Retail      │        20000 │
│         20 │ Wholesale   │    

QUERY-2

In [10]:
df2 = con.sql("""SELECT address.city,sum(OrderAmount) AS TotalRetailSales FROM orders
RIGHT JOIN address
ON orders.CustomerID = address.CustomerID
WHERE orders.TypeOfOrder = 'Retail'
GROUP BY address.City
ORDER BY TotalRetailSales DESC LIMIT 1;""")
df2

┌─────────┬──────────────────┐
│  City   │ TotalRetailSales │
│ varchar │      int128      │
├─────────┼──────────────────┤
│ Kochi   │          4300000 │
└─────────┴──────────────────┘

QUERY-3

In [11]:
df3=con.sql("""SELECT address.city,sum(OrderAmount) AS TotalWholeSales FROM orders
RIGHT JOIN address
ON orders.CustomerID = address.CustomerID
WHERE orders.TypeOfOrder = 'Wholesale'
GROUP BY address.City
ORDER BY TotalWholeSales ASC LIMIT 1;""")
df3

┌─────────┬─────────────────┐
│  City   │ TotalWholeSales │
│ varchar │     int128      │
├─────────┼─────────────────┤
│ Jaipur  │          400000 │
└─────────┴─────────────────┘

QUERY-4

In [12]:
df4 = con.sql("""SELECT A.OrderDate FROM
            (SELECT OrderDate,sum(OrderAmount) AS DaySales FROM orders
			GROUP BY OrderDate
			ORDER BY DaySales DESC LIMIT 1) AS A;""")
df4

┌────────────┐
│ OrderDate  │
│    date    │
├────────────┤
│ 2022-02-02 │
└────────────┘

QUERY-5

In [13]:
df5=con.sql(""" SELECT orders.CustomerID,
    CONCAT(address.Lastname, '', address.Firstname) AS CustomerName,
    address.Address,
    address.City, 
    SUM(orders.OrderAmount) AS TotalOrderValue 
FROM orders
LEFT JOIN address
    ON orders.CustomerID = address.CustomerID
GROUP BY orders.CustomerID, address.Lastname, address.Firstname, address.Address, address.City
ORDER BY orders.CustomerID;
""")
df5

┌────────────┬─────────────────┬────────────────────────┬──────────────────┬─────────────────┐
│ CustomerID │  CustomerName   │        Address         │       City       │ TotalOrderValue │
│  varchar   │     varchar     │        varchar         │     varchar      │     int128      │
├────────────┼─────────────────┼────────────────────────┼──────────────────┼─────────────────┤
│ R2001      │ SharmaSudhanshu │ 12, MG Road, Sector 49 │ Noida            │         3000000 │
│ R2002      │ KhannaKapil     │ 12/5, RR Marg, Bandra  │ Mumbai           │        60000000 │
│ R2003      │ RajMegha        │ 12, T Nagar            │ Chennai          │          200000 │
│ R2004      │ JohnDavid       │ 24, George Nagar       │ Hyderabad        │       200000000 │
│ R2005      │ JamesWilliam    │ 6, Rao Street          │ Kolkotta         │         4200000 │
│ R2006      │ SharmaRaj       │ 7, EM Road             │ Pune             │        10000000 │
│ R2007      │ JacobDaniel     │ 3 c, Ram Marg    

QUERY-6

In [14]:
df6=con.sql("""  
SELECT 
    WeekNumber,TotalSales,
    CASE
        WHEN LAG(TotalSales, 1, 0) OVER (ORDER BY WeekNumber) = 0 THEN NULL
        ELSE (TotalSales - LAG(TotalSales, 1, 0) OVER (ORDER BY WeekNumber)) / LAG(TotalSales, 1, 0) OVER (ORDER BY WeekNumber) * 100
    END AS PercentageChange
FROM   (SELECT WEEK(OrderDate) AS WeekNumber,
        SUM(OrderAmount) AS TotalSales
        FROM orders
        GROUP BY WEEK(OrderDate)) AS  WeeklySales  
ORDER BY WeekNumber;
""")
df6

┌────────────┬────────────┬─────────────────────┐
│ WeekNumber │ TotalSales │  PercentageChange   │
│   int64    │   int128   │       double        │
├────────────┼────────────┼─────────────────────┤
│          5 │  290000000 │                NULL │
│          6 │   80000000 │  -72.41379310344827 │
│          7 │     200000 │              -99.75 │
│          8 │    4000000 │              1900.0 │
│          9 │  210000000 │              5150.0 │
│         11 │     200000 │  -99.90476190476191 │
│         13 │  153900000 │             76850.0 │
│         14 │  152700000 │ -0.7797270955165692 │
│         15 │  140000000 │  -8.316961362148003 │
│         16 │   53400000 │ -61.857142857142854 │
│         17 │  318800000 │  497.00374531835206 │
│         18 │      20000 │  -99.99372647427855 │
│         20 │     600000 │              2900.0 │
│         21 │    6000000 │               900.0 │
│         23 │  160000000 │   2566.666666666667 │
│         25 │   21400000 │             -86.625 │


QUERY-7

In [15]:
df7=con.sql("""  
SELECT  WeekNumber,TotalSales,
    CASE
        WHEN LAG(TotalSales, 1, 0) OVER (ORDER BY WeekNumber) = 0 THEN NULL
        ELSE (TotalSales - LAG(TotalSales, 1, 0) OVER (ORDER BY WeekNumber)) / LAG(TotalSales, 1, 0) OVER (ORDER BY WeekNumber) * 100
    END AS PercentageChange
FROM  
    (SELECT WEEK(OrderDate) AS WeekNumber,
        SUM(OrderAmount) AS TotalSales
    FROM orders
    GROUP BY  WEEK(OrderDate)) AS  WeeklySales  
ORDER BY  WeekNumber;
""")
df7

┌────────────┬────────────┬─────────────────────┐
│ WeekNumber │ TotalSales │  PercentageChange   │
│   int64    │   int128   │       double        │
├────────────┼────────────┼─────────────────────┤
│          5 │  290000000 │                NULL │
│          6 │   80000000 │  -72.41379310344827 │
│          7 │     200000 │              -99.75 │
│          8 │    4000000 │              1900.0 │
│          9 │  210000000 │              5150.0 │
│         11 │     200000 │  -99.90476190476191 │
│         13 │  153900000 │             76850.0 │
│         14 │  152700000 │ -0.7797270955165692 │
│         15 │  140000000 │  -8.316961362148003 │
│         16 │   53400000 │ -61.857142857142854 │
│         17 │  318800000 │  497.00374531835206 │
│         18 │      20000 │  -99.99372647427855 │
│         20 │     600000 │              2900.0 │
│         21 │    6000000 │               900.0 │
│         23 │  160000000 │   2566.666666666667 │
│         25 │   21400000 │             -86.625 │


QUERY-8

In [16]:
df8=con.sql("""  
SELECT 
    WeekNumber,TypeOfOrder, TotalSales,
    CASE
        WHEN LAG(TotalSales, 1, 0) OVER (ORDER BY WeekNumber) = 0 THEN NULL
        ELSE (TotalSales - LAG(TotalSales, 1, 0) OVER (ORDER BY WeekNumber)) / LAG(TotalSales, 1, 0) OVER (ORDER BY WeekNumber) * 100
    END AS PercentageChange
FROM 
    (SELECT WEEK(OrderDate) AS WeekNumber,TypeOfOrder,
     SUM(OrderAmount) AS TotalSales
    FROM orders
	WHERE orders.TypeOfOrder = 'Retail'
    GROUP BY  WEEK(OrderDate),TypeofOrder  
    ) AS  WeeklySales
ORDER BY WeekNumber;
""")
df8

┌────────────┬─────────────┬────────────┬────────────────────┐
│ WeekNumber │ TypeOfOrder │ TotalSales │  PercentageChange  │
│   int64    │   varchar   │   int128   │       double       │
├────────────┼─────────────┼────────────┼────────────────────┤
│          5 │ Retail      │    4000000 │               NULL │
│          7 │ Retail      │     200000 │              -95.0 │
│          8 │ Retail      │    4000000 │             1900.0 │
│         13 │ Retail      │    3700000 │               -7.5 │
│         14 │ Retail      │     700000 │ -81.08108108108108 │
│         16 │ Retail      │    1400000 │              100.0 │
│         17 │ Retail      │    2800000 │              100.0 │
│         18 │ Retail      │      20000 │ -99.28571428571429 │
│         25 │ Retail      │     400000 │             1900.0 │
│         36 │ Retail      │     600000 │               50.0 │
├────────────┴─────────────┴────────────┴────────────────────┤
│ 10 rows                                          4 co

QUERY-9

In [17]:
df9=con.sql("""
  
SELECT 
    WeekNumber,TypeOfOrder,
    TotalSales,
    CASE
        WHEN LAG(TotalSales, 1, 0) OVER (ORDER BY WeekNumber) = 0 THEN NULL
        ELSE (TotalSales - LAG(TotalSales, 1, 0) OVER (ORDER BY WeekNumber)) / LAG(TotalSales, 1, 0) OVER (ORDER BY WeekNumber) * 100
    END AS PercentageChange
FROM 
    (SELECT 
        WEEK(OrderDate) AS WeekNumber,TypeOfOrder,
        SUM(OrderAmount) AS TotalSales
    FROM 
        orders
	WHERE orders.TypeOfOrder = 'Wholesale'
    GROUP BY  WEEK(OrderDate),TypeOfOrder  
    ) AS  WeeklySales
ORDER BY 
    WeekNumber;
""")
df9

┌────────────┬─────────────┬────────────┬─────────────────────┐
│ WeekNumber │ TypeOfOrder │ TotalSales │  PercentageChange   │
│   int64    │   varchar   │   int128   │       double        │
├────────────┼─────────────┼────────────┼─────────────────────┤
│          5 │ Wholesale   │   76000000 │                NULL │
│         11 │ Wholesale   │     200000 │  -99.73684210526315 │
│         13 │ Wholesale   │  150000000 │             74900.0 │
│         14 │ Wholesale   │   62000000 │ -58.666666666666664 │
│         15 │ Wholesale   │   60000000 │  -3.225806451612903 │
│         16 │ Wholesale   │   48000000 │               -20.0 │
│         17 │ Wholesale   │   90000000 │                87.5 │
│         20 │ Wholesale   │     600000 │  -99.33333333333333 │
│         28 │ Wholesale   │     400000 │  -33.33333333333333 │
│         29 │ Wholesale   │    7000000 │              1650.0 │
│         30 │ Wholesale   │    5000000 │  -28.57142857142857 │
│         32 │ Wholesale   │     400000 

QUERY-10

In [18]:
df10=con.sql("""
SELECT DeliveryDate FROM (
SELECT DeliveryDate,
sum(OrderAmount) from orders
GROUP BY DeliveryDate
ORDER BY sum(OrderAmount) ASC LIMIT 1)AS A
;""")
df10

┌──────────────┐
│ DeliveryDate │
│     date     │
├──────────────┤
│ 2022-05-04   │
└──────────────┘

QUERY-11

In [19]:
df11=con.sql("""
SELECT CustomerID,FullName,OrderAmount FROM
		(SELECT orders.CustomerID,CONCAT(address.LastName,'',address.FirstName) AS FullName,
        sum(OrderAmount) AS OrderAmount,
        month(OrderDate) AS MonthNumber,
        (Rank() OVER (ORDER BY sum(OrderAmount) DESC)) AS Ranks
		FROM orders
        LEFT JOIN address
        ON orders.CustomerID=address.CustomerID
		WHERE month(OrderDate) = 6
		GROUP BY orders.CustomerID,month(OrderDate),FullName) AS A
WHERE A.Ranks =1;""")
df11

┌────────────┬────────────┬─────────────┐
│ CustomerID │  FullName  │ OrderAmount │
│  varchar   │  varchar   │   int128    │
├────────────┼────────────┼─────────────┤
│ R2037      │ GuptaReema │   160000000 │
└────────────┴────────────┴─────────────┘

QUERY-12

In [20]:
df12=con.sql("""
SELECT TypeOfOrder,OrderAmount FROM
		(SELECT orders.TypeOfOrder,
        sum(OrderAmount) AS OrderAmount,
        month(OrderDate) AS MonthNumber,
        (Rank() OVER (ORDER BY sum(OrderAmount) DESC)) AS Ranks
		FROM orders
		WHERE month(OrderDate) = 6
		GROUP BY month(OrderDate), orders.TypeOfOrder) AS A
WHERE A.Ranks =1;
""")
df12

┌─────────────┬─────────────┐
│ TypeOfOrder │ OrderAmount │
│   varchar   │   int128    │
├─────────────┼─────────────┤
│ Overseas    │   181000000 │
└─────────────┴─────────────┘

In [22]:
con.close()